In [1]:
from collections import deque, namedtuple
import itertools

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

raw_input = data.strip()

https://adventofcode.com/2024/day/9

In [3]:
## Part 1
# Numbers alternate between file and free space
# The number is the number of blocks in the file/hole
# Each file also has an ID starting from 0. This is the file number in the original ordering (ignoring holes)
File = namedtuple("File", ["is_hole", "blocks", "id"])
starting_files = []
file_count = 0
is_holes = itertools.cycle([False, True])
for blocks, is_hole in zip(raw_input, is_holes):
    if is_hole:
        file_id = None
    else:
        file_id = file_count
        file_count += 1
    starting_files.append(File(is_hole, int(blocks), file_id))

file_count

10

In [4]:
def visualise(files: list[File]) -> str:
    # Just print the last digit of the ID for now (only used for small problems n<=10)
    return "".join(("." if file.is_hole else str(file.id % 10)) * file.blocks for file in files)

def checksum(files: list[File]) -> int:
    out = 0
    pos = 0
    for file in files:
        if not file.is_hole:
            # out += file.id * sum((pos + i) for i in range(file.blocks))
            # Off-by-one errors below!
            # out += file.id * sum(range(pos, pos + file.blocks))  # Note: sum(range) doesn't shortcut like `n in range()` does
            positions = ((pos + pos + file.blocks - 1) * file.blocks) // 2  # (start + end) * n / 2
            out += file.id * positions
        pos += file.blocks
    return out

In [5]:
# Compress the filesystem by moving blocks from the right side to fill holes in the left
compressed = []
pending_files = deque(starting_files)
try:
    while pending_files:
        file = pending_files.popleft()
        if not file.is_hole:
            compressed.append(file)
            continue
        # If it's a hole, consume files from the right side until the hole is filled
        hole_size = file.blocks
        while hole_size > 0:
            # Get the next file from the right side. If none left (IndexError), we're done!
            right_file = pending_files.pop()
            if right_file.is_hole:
                # A hole can't fill a hole. Skip.
                continue
            elif right_file.blocks >= hole_size:
                # Right file fills hole completely
                compressed.append(File(False, hole_size, right_file.id))
                # Put remaining blocks back on the right side
                if (remaining_blocks := right_file.blocks - hole_size) > 0:
                    pending_files.append(File(False, remaining_blocks, right_file.id))
                hole_size = 0
            else:
                # Hole not fully-filled
                hole_size -= right_file.blocks
                compressed.append(right_file)
except IndexError:
    pass

checksum(compressed)

1928

In [6]:
## Part 2
# Compress without splitting files instead
# Attempt to move each file exactly once from right-to-left
#  Move it to the leftmost hole that can fit it. If none can fit it, don't move

# I think a greedy left-to-right approach could still work, we just need to scan from the right until we find a file that fits in the hole
# But since we're keeping the spaces, a deque is less useful
# Other options: pending_files {id: (File, start, end)} and holes [(start, end, blocks)]
# - sort holes by start index
# - ...


In [7]:
# Brute force (very inefficient! 32 sec on input)
# - This scans files on the right side many times. It would be more efficient to keep track of the "next" file of each blocksize
defragged = []
pending_files = deque(starting_files)

while pending_files:
    file = pending_files.popleft()
    if not file.is_hole:
        defragged.append(file)
        continue
    # If it's a hole, try to consume files from the right side until the hole is filled
    hole_size = file.blocks
    right_files_skipped = []
    while pending_files:
        # Get the next file from the right side
        right_file = pending_files.pop()
        if right_file.is_hole:
            # A hole can't fill a hole. Skip.
            right_files_skipped.append(right_file)
            continue
        elif right_file.blocks <= hole_size:
            # Right file fits in the hole
            defragged.append(right_file)
            hole_size -= right_file.blocks
            # Put a hole in pending list where right_file was
            right_files_skipped.append(File(is_hole=True, blocks=right_file.blocks, id=None))
            if hole_size <= 0:
                # Hole completely filled, we're done!
                break
            # Note: we don't need to check skipped files again for this hole, since the hole's only gotten smaller
        else:
            # Right file doesn't fit, skip.
            right_files_skipped.append(right_file)
            continue
 
    # No files left. Remaining hole stays in result
    if hole_size > 0:
        defragged.append(File(is_hole=True, blocks=hole_size, id=None))
    # Put skipped files back in the deque
    pending_files.extend(right_files_skipped[::-1])
    

checksum(defragged)

2858